In [1]:
import tensorflow  as tf
import datetime

In [2]:
tf.__version__

'2.0.0'

In [3]:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

In [4]:
train_image.shape

(60000, 28, 28)

In [5]:
train_image = tf.expand_dims(train_image, -1)
test_image = tf.expand_dims(test_image, -1)

In [6]:
train_image.shape, test_image.shape

(TensorShape([60000, 28, 28, 1]), TensorShape([10000, 28, 28, 1]))

In [7]:
train_image = tf.cast(train_image/255, tf.float32)
test_image = tf.cast(test_image/255, tf.float32)

In [8]:
train_label = tf.cast(train_label, tf.int64)
test_label = tf.cast(test_label, tf.int64)

In [9]:
dataset_train = tf.data.Dataset.from_tensor_slices((train_image, train_label))
dataset_test = tf.data.Dataset.from_tensor_slices((test_image, test_label))

In [10]:
dataset_train, dataset_test

(<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>,
 <TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>)

In [11]:
dataset_train = dataset_train.repeat().shuffle(60000).batch(128)
dataset_test = dataset_test.batch(128)

In [12]:
dataset_train

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], input_shape = (None, None, 1), activation='relu'),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [14]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
import os

In [16]:
log_dir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))

In [17]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

In [18]:
file_writer = tf.summary.create_file_writer(log_dir + '/lr')

In [19]:
file_writer.set_as_default

<bound method ResourceSummaryWriter.set_as_default of <tensorflow.python.ops.summary_ops_v2.ResourceSummaryWriter object at 0x63de1fc10>>

In [20]:
def lr_sche(epoch):
    learing_rate = 0.2
    if epoch > 5:
        learing_rate = 0.02
    if epoch > 10:
        learing_rate = 0.01
    if epoch > 20:
        learing_rate = 0.005
    tf.summary.scalar('leaning_rate', data=learing_rate, step=epoch)
    return learing_rate

In [21]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)

In [22]:
model.fit(
    dataset_train, 
    epochs=25, 
    steps_per_epoch=60000//128, 
    validation_data=dataset_test, 
    validation_steps=10000//128, 
    callbacks=[tensorboard_callback, lr_callback]
)

Train for 468 steps, validate for 78 steps
Epoch 1/25
468/468 [==============================] - 22s 48ms/step - loss: 2.3722 - accuracy: 0.1003 - val_loss: 2.3105 - val_accuracy: 0.1032
Epoch 2/25
468/468 [==============================] - 21s 45ms/step - loss: 2.3132 - accuracy: 0.1009 - val_loss: 2.3129 - val_accuracy: 0.0981
Epoch 3/25
468/468 [==============================] - 25s 53ms/step - loss: 2.3128 - accuracy: 0.1019 - val_loss: 2.3075 - val_accuracy: 0.1135
Epoch 4/25
468/468 [==============================] - 21s 45ms/step - loss: 2.3122 - accuracy: 0.1039 - val_loss: 2.3070 - val_accuracy: 0.0982
Epoch 5/25
468/468 [==============================] - 19s 42ms/step - loss: 2.3135 - accuracy: 0.1018 - val_loss: 2.3149 - val_accuracy: 0.0981
Epoch 6/25
468/468 [==============================] - 21s 45ms/step - loss: 2.3122 - accuracy: 0.1043 - val_loss: 2.3041 - val_accuracy: 0.1029
Epoch 7/25
468/468 [==============================] - 20s 42ms/step - loss: 2.3025 - accuracy

In [23]:
%load_ext tensorboard
%matplotlib inline

In [24]:
%tensorboard --logdir logs